In [1]:
import pandas as pd
import numpy as np
import random
import copy
import math
from collections import defaultdict

randomlist = []
df_demand = {}
random.seed(2) # Use the same random seed
for i in range(0,20):   # 20 depots
    n = random.randint(50,150) # Depot Capacity
    randomlist.append(n)

for i in range(20):
    df_demand["D"+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in df_demand.items():
    sum = sum + v

df_supply = {}
for i in range(sum):
    df_supply[i] = 1.0

np.random.seed(2) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,20, size=(sum,20))).astype(float)

random.seed(2) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)

age = {}
for i in range(len(age_b)):
    age[i] = age_b[i]

kpl = []
co2 = []
random.seed(2) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)

fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])

random.seed(2) # Use the same random seed
doc = {}
for i in range(0,20):
    n = random.uniform(50,100)
    doc[i] = n

df_tdkom_c = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom_c[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom_c


df_cost = df_tdkom_c.copy(deep=True)

# Calculating the row deadkm opportunity matrix 
df_rowmin = df_cost.min(axis=1)
df_rowopp = df_cost.copy(deep=True)
for i in range(0,len(df_cost)):
    for j in range(0,len(df_cost.columns)):
        df_rowopp.iloc[i,j] = df_rowopp.iloc[i,j] - df_rowmin.iloc[i]

# Calculating the column deadkm opportunity matrix 
df_colmin = df_cost.min(axis=0)
df_colopp = df_cost.copy(deep=True)
for i in range(0,len(df_cost.columns)):
    for j in range(0,len(df_cost)):
        df_colopp.iloc[j,i] = df_colopp.iloc[j,i] - df_colmin.iloc[i]

# Calculating the total deadkm opportunity matrix 
df_tdkom = round(df_rowopp + df_colopp, 2)
#print("Total Deadkm opportunity matrix is:")
#print(df_tdkom)

df_tdkom_c.columns = [("D"+str(i)) for i in range(1,21)]
dm = df_tdkom_c.transpose()
costs_l = dm.to_dict()
costs2=copy.deepcopy(costs_l)

df_tdkom.columns = [("D"+str(i)) for i in range(1,21)]
dk = df_tdkom.transpose()
costs_t = dk.to_dict()
cols = sorted(df_demand.keys())
costs1=copy.deepcopy(costs_t)

In [2]:
res = dict((k, defaultdict(int)) for k in costs_t)
g = {}
for x in df_supply:
    g[x] = sorted(costs_t[x].keys(), key=lambda g: costs_t[x][g])
for x in df_demand:
    g[x] = sorted(costs_t.keys(), key=lambda g: costs_t[g][x])
# print(costs1)
flag=0
while g:
    l=[]
    for x in df_demand:

        # print(demand[x],supply[g[x][0]])
        if df_demand[x]> df_supply[g[x][0]]:
            l.append(x)
            if len(l)==2:
                break
    # break
    mi=100000
    dem="D"
    # print(l)
    if len(l)==0:
        for x in df_demand:
            t=g[x][0]
            res[t][x]+=df_demand[x]
            df_supply[t]-=df_demand[x]
            df_demand[x]=0
            if df_supply[t]==0:
                if df_supply[t] == 0:
                    for k, n in df_demand.items():
                        if n != 0:
                            g[k].remove(t)
                del g[t]
                del df_supply[t]
            for k, n in df_supply.items():
                if n != 0:
                    g[k].remove(x)
            del g[x]
            del df_demand[x]
            break
        continue
    for x in l:

        diff = costs_t[g[x][1]][x]-costs_t[g[x][0]][x] if len(g[x])>1 else costs_t[g[x][0]]
        if diff < mi:
            mi=diff
            dem=x
    x=dem 

    # print(x,mi)
    # break

    while(df_demand[x]>0):

        v=min(df_demand[x],df_supply[g[x][0]])
        df_demand[x]-=v
        df_supply[g[x][0]]-=v
        res[g[x][0]][x]+=v
        t=g[x][0]
        # print(t)

        if df_supply[t]==0:
            if df_supply[t] == 0:
                for k, n in df_demand.items():
                    if n != 0:
                        g[k].remove(t)
            del g[t]
            del df_supply[t]
    for k, n in df_supply.items():
        if n != 0:
                g[k].remove(x)
    del g[x]
    del df_demand[x]


# for x in demand:

# 	t= g[x][0]
# 	if demand[x]> supply[t]:
# 		print("Help")
# 	res[t][x]+= demand[x] 
# 	demand[x]=0
# 	supply[t]-= demand[x]

# 	if supply[t]==0:
# 		if supply[t] == 0:
# 			for k, n in demand.items():
# 				if n != 0:
# 					g[k].remove(t)
# 		del g[t]
# 		del supply[t]
# 	for k, n in supply.items():
# 		if n != 0:
# 			g[k].remove(x)
# 	del g[x]





#print(df_demand,df_supply)	

# print(costs1)
cost = 0
for g in sorted(costs2):
    # print (g, " ",)
    for n in cols:
        y = res[g][n]
        if y != 0:
            pass
            # print (y,)
        cost += y * costs2[g][n]
        # print ("  ",)
    # print(" ")
# print ("Total Cost = ", cost)


In [3]:
cost

276141.96581715444